In [1]:
cd ..

/home/avani.gupta/bpirl2


In [2]:
pwd

'/home/avani.gupta/bpirl2'

In [3]:
from utils import calc_third_quartile, get_unique_act, get_compliant_cases 
import numpy as np
import os
import pickle
import pandas as pd
import random
from statistics import mean, median

In [4]:
df2 = pd.read_pickle('dataset/preprocessed/bpi_12_w_design_mat.pkl')

In [5]:
df2

,0,1,2,3,4,5,6,duration_time,remaining_time,class,CaseID
0,0,0,0,1,0,0,0,0.000000,11.952778,3,173688
1,0,0,0,0,0,1,0,0.022222,11.930556,5,173688
2,0,0,0,0,0,1,0,7.177083,4.753472,5,173688
3,0,0,0,0,0,1,0,1.792361,2.961111,5,173688
4,0,0,0,0,0,0,1,2.961111,0.000000,6,173688
...,...,...,...,...,...,...,...,...,...,...,...
72408,0,1,0,0,0,0,0,0.000000,0.000000,1,214370
72409,0,1,0,0,0,0,0,0.000000,9.140972,1,214373
72410,0,0,0,1,0,0,0,0.454861,8.686111,3,214373
72411,0,0,0,0,0,1,0,8.686111,0.000000,5,214373


In [6]:
# get process flow compliance cases only
df = get_compliant_cases(df2,dset="bpi_12_w")

In [7]:
df

,0,1,2,3,4,5,6,duration_time,remaining_time,class,CaseID
0,0,0,0,1,0,0,0,0.000000,11.952778,3,173688
1,0,0,0,0,0,1,0,0.022222,11.930556,5,173688
2,0,0,0,0,0,1,0,7.177083,4.753472,5,173688
3,0,0,0,0,0,1,0,1.792361,2.961111,5,173688
4,0,0,0,0,0,0,1,2.961111,0.000000,6,173688
...,...,...,...,...,...,...,...,...,...,...,...
72406,0,0,0,0,0,1,0,0.021528,7.009722,5,214364
72407,0,0,0,0,0,1,0,7.009722,0.000000,5,214364
72409,0,1,0,0,0,0,0,0.000000,9.140972,1,214373
72410,0,0,0,1,0,0,0,0.454861,8.686111,3,214373


In [8]:
dat_group = df.groupby("CaseID")

total_iter = len(dat_group.ngroup())
case_duration_dic = {}
for name, gr in dat_group:
    case_duration_dic[name] = gr['duration_time'].sum()
    

In [9]:
max(df['duration_time'])

30.92222222222222

In [10]:
case_duration_dic

{173688: 11.95277777777778,
 173691: 9.106944444444444,
 173694: 34.229166666666664,
 173703: 0.05694444444444444,
 173706: 0.19722222222222224,
 173709: 30.991666666666667,
 173712: 2.188888888888889,
 173715: 11.971527777777778,
 173718: 25.940277777777776,
 173721: 9.213888888888889,
 173730: 19.800694444444446,
 173736: 16.847916666666666,
 173739: 13.038194444444446,
 173742: 13.038888888888888,
 173745: 6.891666666666667,
 173748: 2.0555555555555554,
 173751: 9.79513888888889,
 173760: 12.98611111111111,
 173766: 5.1715277777777775,
 173775: 0.05416666666666667,
 173784: 43.18819444444445,
 173787: 16.38888888888889,
 173790: 7.205555555555556,
 173793: 15.080555555555556,
 173796: 7.327083333333333,
 173799: 1.371527777777778,
 173805: 31.026388888888885,
 173811: 31.045833333333334,
 173814: 0.08680555555555555,
 173817: 7.974305555555556,
 173823: 9.985416666666667,
 173835: 15.177083333333334,
 173841: 1.1486111111111112,
 173844: 8.198611111111111,
 173847: 9.020833333333332

reference for calulating quartile [here](http://web.mnstate.edu/peil/MDEV102/U4/S36/S363.html#:~:text=The%20third%20quartile%2C%20denoted%20by,25%25%20lie%20above%20Q3%20)

In [11]:
mean(case_duration_dic.values())

15.718907135529657

In [12]:
# quartile calculation
import statistics
def calc_third_quartile(lis):
    lis.sort()
    size = len(lis)
    lis_upper_half = lis[size//2:-1]
    third_quartile = statistics.median(lis_upper_half)
    return third_quartile
    
case_durations = list(case_duration_dic.values())
third_quartile  = calc_third_quartile(case_durations)

In [13]:
third_quartile

24.00138888888889

### Filter dataset for RL model


In [15]:
cases_gs = []
cases_gv = []
for k,v in case_duration_dic.items():
    if v <= third_quartile:
        cases_gs.append(k)
    else:
        cases_gv.append(k)

In [16]:
len(cases_gs), len(cases_gv)

(4955, 1652)

In [17]:
tot = len(cases_gs)+ len(cases_gv)
percent_gs_cases = len(cases_gs) / tot
print(percent_gs_cases)

0.749962161344029


In [18]:
cases_train = cases_gs
cases_test = cases_gv


In [19]:
df.shape, len(cases_train),  len(cases_test)

((64171, 11), 4955, 1652)

In [20]:
data_train = df.loc[df['CaseID'].isin(cases_train)]
data_test = df.loc[df['CaseID'].isin(cases_test)]

In [21]:
data_train

,0,1,2,3,4,5,6,duration_time,remaining_time,class,CaseID
0,0,0,0,1,0,0,0,0.000000,11.952778,3,173688
1,0,0,0,0,0,1,0,0.022222,11.930556,5,173688
2,0,0,0,0,0,1,0,7.177083,4.753472,5,173688
3,0,0,0,0,0,1,0,1.792361,2.961111,5,173688
4,0,0,0,0,0,0,1,2.961111,0.000000,6,173688
...,...,...,...,...,...,...,...,...,...,...,...
72406,0,0,0,0,0,1,0,0.021528,7.009722,5,214364
72407,0,0,0,0,0,1,0,7.009722,0.000000,5,214364
72409,0,1,0,0,0,0,0,0.000000,9.140972,1,214373
72410,0,0,0,1,0,0,0,0.454861,8.686111,3,214373


In [22]:
data_test

,0,1,2,3,4,5,6,duration_time,remaining_time,class,CaseID
14,0,0,0,1,0,0,0,0.000000,34.229167,3,173694
15,0,0,0,1,0,0,0,1.990278,32.238889,3,173694
16,0,0,0,1,0,0,0,0.090972,32.147917,3,173694
17,0,0,0,1,0,0,0,0.005556,32.142361,3,173694
18,0,0,0,0,0,1,0,0.002778,32.139583,5,173694
...,...,...,...,...,...,...,...,...,...,...,...
66962,0,0,0,1,0,0,0,0.000000,24.862500,3,210626
66963,0,0,0,0,0,1,0,9.967361,14.895139,5,210626
66964,0,0,0,0,0,1,0,1.097917,13.797222,5,210626
66965,0,0,0,0,0,1,0,5.788194,8.009028,5,210626


## Analysing unique events

In [23]:
a = get_unique_act(data_train)

In [24]:
len(a)

6

In [25]:
tot = get_unique_act(df)
len(tot)

6

In [26]:
lis = []
for act in tot:
    if act not in a:
        lis.append(act)
lis        

[]

In [27]:
for act in lis:
    df_sub = df[df["class"] == act]
    caseid_lis = list(df_sub["CaseID"])
    l = len(caseid_lis)
    caseid_sel = caseid_lis[:l//2]
    if len(caseid_sel) == 0:
        caseid_sel = caseid_lis
        
    r = df.loc[df['CaseID'].isin(caseid_sel)]
    data_train = data_train.append(r)


In [28]:
data_train

,0,1,2,3,4,5,6,duration_time,remaining_time,class,CaseID
0,0,0,0,1,0,0,0,0.000000,11.952778,3,173688
1,0,0,0,0,0,1,0,0.022222,11.930556,5,173688
2,0,0,0,0,0,1,0,7.177083,4.753472,5,173688
3,0,0,0,0,0,1,0,1.792361,2.961111,5,173688
4,0,0,0,0,0,0,1,2.961111,0.000000,6,173688
...,...,...,...,...,...,...,...,...,...,...,...
72406,0,0,0,0,0,1,0,0.021528,7.009722,5,214364
72407,0,0,0,0,0,1,0,7.009722,0.000000,5,214364
72409,0,1,0,0,0,0,0,0.000000,9.140972,1,214373
72410,0,0,0,1,0,0,0,0.454861,8.686111,3,214373


In [29]:
len(get_unique_act(data_train)), len(get_unique_act(data_test))

(6, 6)

In [30]:
len(get_unique_act(df))

6

In [31]:
env_name = "bpi_12_w"
name = env_name+'_d0'
pickle.dump(data_train, open(name+"_train_RL.pkl", "wb"))
pickle.dump(data_test, open(name+"_test_RL.pkl", "wb"))